In [1]:
from preprocessing import *
from proj2_helpers import *
from UNet_model import *
from run import *


In [2]:
X_train, X_test, Y_train, Y_test = preprocess(divide_set=True, ratio=0.25)



processing image  0 / 100
processing image  1 / 100
processing image  2 / 100
processing image  3 / 100
processing image  4 / 100
processing image  5 / 100
processing image  6 / 100
processing image  7 / 100
processing image  8 / 100
processing image  9 / 100
processing image  10 / 100
processing image  11 / 100
processing image  12 / 100
processing image  13 / 100
processing image  14 / 100
processing image  15 / 100
processing image  16 / 100
processing image  17 / 100
processing image  18 / 100
processing image  19 / 100
processing image  20 / 100
processing image  21 / 100
processing image  22 / 100
processing image  23 / 100
processing image  24 / 100
processing image  25 / 100
processing image  26 / 100
processing image  27 / 100
processing image  28 / 100
processing image  29 / 100
processing image  30 / 100
processing image  31 / 100
processing image  32 / 100
processing image  33 / 100
processing image  34 / 100
processing image  35 / 100
processing image  36 / 100
processing 

In [3]:
X_train = np.stack(X_train, axis = 0)
X_test = np.stack(X_test, axis = 0)
Y_train = np.stack(Y_train, axis = 0)
Y_test = np.stack(Y_test, axis = 0)

In [4]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(225, 400, 400, 3)
(75, 400, 400, 3)
(225, 400, 400, 1)
(75, 400, 400, 1)


In [ ]:
import importlib
importlib.reload(run)

In [16]:
def run__(X_train, Y_train, X_test, Y_test):

    IMG_HEIGHT = IMG_WIDTH = 400
    IMG_CHANNELS = 3

    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs)
    print("HELP ! ", inputs, " o")
    print("HELP ! ", s, " o")
    """unet_model = build_unet(s, n_filters=16, dropout_down=0.05, dropout_up=0.1, batch_norm=True,
                            activation_fct='relu', final_activation='sigmoid', kernel_size=(3, 3))

    unet_model.fit(X_train, Y_train, validation_split=0.1,
                   batch_size=16, epochs=25)
    results = unet_model.fit(X_train, Y_train, batch_size=16, epochs=25)

    # fitting the model to the train data
    # evaluating performance of the model
    result = unet_model.evaluate(X_test, Y_test)"""
    return 0


In [17]:
tf.keras.backend.clear_session()
result = run__(X_train, Y_train, X_test, Y_test)

HELP !  Tensor("input_1:0", shape=(None, 400, 400, 3), dtype=float32)  o
HELP !  Tensor("lambda/truediv:0", shape=(None, 400, 400, 3), dtype=float32)  o


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(img_aug_arr, patch_size, patch_size) for img_aug_arr in imgs_aug_arr]
gt_patches = [img_crop(gt_img_aug_arr, patch_size, patch_size) for gt_img_aug_arr in gt_imgs_aug_arr]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [ ]:
img_patches.shape

In [ ]:
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [ ]:
print('Computed ' + str(X.shape[0]) + ' features')
print('Feature dimension = ' + str(X.shape[1]))
print('Number of classes = ' + str(np.max(Y)))  #TODO: fix, length(unique(Y)) 

Y0 = [i for i, j in enumerate(Y) if j == 0]
Y1 = [i for i, j in enumerate(Y) if j == 1]
print('Class 0: ' + str(len(Y0)) + ' samples')
print('Class 1: ' + str(len(Y1)) + ' samples')

from sklearn import linear_model

# we create an instance of the classifier and fit the data
logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced")
logreg.fit(X, Y)

# Predict on the training set
Z = logreg.predict(X)

# Get non-zeros in prediction and grountruth arrays
Zn = np.nonzero(Z)[0]
Yn = np.nonzero(Y)[0]

TPR = len(list(set(Yn) & set(Zn))) / float(len(Z))
print('True positive rate = ' + str(TPR))